In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.static import teams 
import helper_functions as hf
from IPython.display import clear_output
import os

In [2]:
def get_season(season, season_type):
    season_i_teams = leaguegamelog.LeagueGameLog(season = str(i), season_type_all_star=season_type).get_data_frames()[0]
    season_i_players = leaguegamelog.LeagueGameLog(season = str(i), player_or_team_abbreviation = 'P', season_type_all_star=season_type).get_data_frames()[0]
    season_i_teams['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    season_i_players['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    return season_i_teams, season_i_players

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
teams_list = teams.get_teams()

teams_per = dict()

for team in teams_list:
    team_id = team['id']
    teams_per[team_id] = 0

seasons_teams = []
seasons_players = []
first_season = 2008
last_season = 2021
first_season_id = 20000 + first_season

print("Getting NBA Seasons Information...")
for i in range(first_season,last_season):
    season_i_teams, season_i_players = get_season(str(i), 'Regular Season')
    season_i_teams_playoffs, season_i_players_playoffs = get_season(str(i), 'Playoffs')
    
    season_i_teams = reduce(lambda  left,right: pd.merge(left,right, how='outer'), [season_i_teams, season_i_teams_playoffs])
    season_games_plyrs = reduce(lambda  left,right: pd.merge(left,right, how='outer'), [season_i_players, season_i_players_playoffs])
    
    seasons_teams.append(season_i_teams)
    seasons_players.append(season_i_players)
    print("{}/{}".format(i,last_season))


dfs = []

print("Cleaning the data...")

season_games = reduce(lambda  left,right: pd.merge(left,right, how='outer'), seasons_teams)
season_games_plyrs = reduce(lambda  left,right: pd.merge(left,right, how='outer'), seasons_players)
season_games.dropna(subset=['FG_PCT','FT_PCT','FG3_PCT'], inplace=True)

season_games_plyrs['GAME_ID'] = pd.to_numeric(season_games_plyrs['GAME_ID'])
season_games['GAME_ID'] = pd.to_numeric(season_games['GAME_ID'])
season_games['GAME_DATE'] = pd.to_datetime(season_games['GAME_DATE'])
season_games_plyrs['GAME_DATE'] = pd.to_datetime(season_games_plyrs['GAME_DATE'])

print('size', len(season_games.index))

Getting NBA Seasons Information...


In [ ]:
print("Initializing ELOs...")

elo_dic = dict()

for team in teams_list:
    elo_dic[team['id']] = 1500

matches_organized = []
matches_organized_playoffs = []

n_last_games = 10
n_last_specific_games = 5

season_id = ''    
print('Getting historical odds...')
odds = hf.load_bets_csv()

In [ ]:
season_i_teams_playoffs.tail()

In [ ]:
season_games.head()

In [ ]:
def get_match_info(is_away, game_info, stats_team_a, stats_team_b, winner):
    if is_away:
        return (game_info + stats_team_b + stats_team_a + [1 if winner == 'B' else 0])
    else:
        return (game_info + stats_team_a + stats_team_b + [1 if winner == 'A' else 0])

In [ ]:
print("Creating CSV file of all games...")
for i, g in season_games.groupby(season_games.index // 2):
    clear_output(wait=True)
    print("{}/{}".format(i, len(season_games.index) // 2))
    if g.iloc[[0],:].iloc[0]['WL'] == None:
        break

    if season_id != '' and season_id != hf.get_season_year(g.iloc[[0],:].iloc[0]['SEASON_ID']):
        hf.reset_season_elo(elo_dic)

    season_id = hf.get_season_year(g.iloc[[0],:].iloc[0]['SEASON_ID'])
        
    game_id = g.iloc[[0],:].iloc[0]['GAME_ID']
    game_date = g.iloc[[0],:].iloc[0]['GAME_DATE']

    team_a_id = g.iloc[[0],:].iloc[0]['TEAM_ID']
    team_b_id = g.iloc[1:2,:].iloc[0]['TEAM_ID']

    team_a_abbv = g.iloc[[0],:].iloc[0]['TEAM_ABBREVIATION']
    team_b_abbv = g.iloc[1:2,:].iloc[0]['TEAM_ABBREVIATION']
    
    winner = 'A' if g.iloc[[0],:].iloc[0]['WL'] == 'W' else 'B'
    
    # Update ELO after stats computed
    team_a_pts = g.iloc[[0],:].iloc[0]['PTS']
    team_b_pts = g.iloc[1:2,:].iloc[0]['PTS']
    elo_a = elo_dic[team_a_id]
    elo_b = elo_dic[team_b_id]

    if '@' in g.iloc[[0],:].iloc[0]['MATCHUP']:
        team_b_odds, team_a_odds = hf.get_teams_odds(team_b_id, team_a_id, game_date, odds)
    else:
        team_a_odds, team_b_odds = hf.get_teams_odds(team_a_id, team_b_id, game_date, odds)

    team_a_previous_games = season_games.loc[(season_games['TEAM_ID'] == team_a_id) & (season_games['GAME_DATE'] < game_date)]
    team_b_previous_games = season_games.loc[(season_games['TEAM_ID'] == team_b_id) & (season_games['GAME_DATE'] < game_date)]
    team_a_season_games = team_a_previous_games.loc[hf.get_season_year(team_a_previous_games['SEASON_ID']) == season_id]
    team_b_season_games = team_b_previous_games.loc[hf.get_season_year(team_b_previous_games['SEASON_ID']) == season_id]

    # Getting teams last 10 games
    team_a_previous_n_games = team_a_season_games.iloc[-n_last_games:,:]
    team_b_previous_n_games = team_b_season_games.iloc[-n_last_games:,:]

    if len(team_a_previous_games.index) > 0:
        if team_a_previous_games.iloc[-1]['GAME_ID'] == g.iloc[[0],:].iloc[0]['GAME_ID']:
            break
            
    if not g.iloc[[0],:].iloc[0]['IS_PLAYOFFS']:
        print('Not a playoff game')
        hf.update_elo(winner, elo_a, elo_b, elo_dic, team_a_id, team_b_id, team_a_pts, team_b_pts)
        continue

    if not (int(season_id) >= hf.get_season_year(first_season_id) and len(team_a_previous_n_games.index) >= 5 and len(team_b_previous_n_games.index) >= 5 and team_a_odds != None and team_b_odds != None):
        print("Not enough games.")
        hf.update_elo(winner, elo_a, elo_b, elo_dic, team_a_id, team_b_id, team_a_pts, team_b_pts)
        continue

    # Getting player information
    teams_per[team_a_id] = hf.get_team_per_mean(team_a_id, game_id, game_date, season_id, season_games_plyrs)
    teams_per[team_b_id] = hf.get_team_per_mean(team_b_id, game_id, game_date, season_id, season_games_plyrs)

    # Season Win Percentage
    team_a_season_pct = hf.get_wl_pct(team_a_season_games)[0]
    team_b_season_pct = hf.get_wl_pct(team_b_season_games)[0]

    # Poins Conceded
    team_a_previous_games_pts_conceded = hf.team_points_conceded(team_a_previous_n_games, season_games)
    team_b_previous_games_pts_conceded = hf.team_points_conceded(team_b_previous_n_games, season_games)

    stats_team_a = hf.get_team_stats (team_a_previous_n_games, team_a_previous_games_pts_conceded, team_a_season_pct, teams_per[team_a_id], team_a_odds, elo_a)
    stats_team_b = hf.get_team_stats (team_b_previous_n_games, team_b_previous_games_pts_conceded, team_b_season_pct, teams_per[team_b_id], team_b_odds, elo_b)

    match_info = get_match_info('@' in g.iloc[[0],:].iloc[0]['MATCHUP'], [season_id, game_date, team_b_abbv, team_a_abbv], stats_team_a, stats_team_b, winner)
    if g.iloc[[0],:].iloc[0]['IS_PLAYOFFS']:
        print("{} {}".format(elo_a, elo_b))
        print(match_info)
        matches_organized_playoffs.append(match_info)
        break
    else:
        matches_organized.append(match_info)

    hf.update_elo(winner, elo_a, elo_b, elo_dic, team_a_id, team_b_id, team_a_pts, team_b_pts)

In [ ]:
print(len(matches_organized_playoffs))

In [ ]:
def parse_df_to_csv(dataframe, columns, path, filename):
    if not os.path.exists(path):
        os.makedirs(path)
    final_df = pd.DataFrame(dataframe, columns=columns)
    final_df.to_csv("{}/{}".format(path, filename))

In [ ]:
columns = ['SEASON_ID', 'GAME_DATE', 'TEAM_A', 'TEAM_B',
           'PTS_A', 'PTS_CON_A', 'FG_PCT_A', 'FG3_PCT_A', 'FT_PCT_A', 'REB_A', 'TOV_A', 'SEASON_A_PCT', 'PER_A', 'ODDS_A', 'ELO_A',
           'PTS_B', 'PTS_CON_B', 'FG_PCT_B', 'FG3_PCT_B', 'FT_PCT_B', 'REB_B', 'TOV_B', 'SEASON_B_PCT', 'PER_B', 'ODDS_B', 'ELO_B',
           'WINNER']
# parse_df_to_csv(matches_organized, columns, '../data', '{}-{}.csv'.format(first_season, last_season-1))
parse_df_to_csv(matches_organized_playoffs, columns, '../data/playoffs', '{}-{}.csv'.format(first_season, last_season-1))